In [ ]:
import tensorflow as tf
import numpy as np
import os
from pycocotools.coco import COCO
import cv2
import matplotlib.pyplot as plt

from utils import save_numpy_array

In [ ]:
# Paths to COCO dataset
instances_val = r"..\datasets\coco"

COCO_PATH = instances_val
TRAIN_IMAGES_PATH = os.path.join(COCO_PATH, 'images', 'val2017')
ANNOTATIONS_PATH = os.path.join(COCO_PATH, 'annotations', 'instances_val2017.json')
OUTPUT_FILES =  os.path.join(COCO_PATH, 'preprocessed', 'val2017')   

OUTPUT_PATH = os.path.abspath(OUTPUT_FILES)

In [ ]:
# Initialize COCO
coco = COCO(ANNOTATIONS_PATH)

# Step 1: Analyze Class Representation and Get Top 5 Classes
category_counts = {}
for ann in coco.dataset['annotations']:
    cat_id = ann['category_id']
    category_counts[cat_id] = category_counts.get(cat_id, 0) + 1

# Get category names
categories = {cat['id']: cat['name'] for cat in coco.dataset['categories']}

# Get the top 5 categories
sorted_categories = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)
top_10_categories = sorted_categories[:2]
top_10_category_ids = [cat[0] for cat in top_10_categories]

print("Top 10 Categories:", [categories[cat_id] for cat_id in top_10_category_ids])

In [ ]:

def mask_generator(class_id,image_id, display = True):
    coco = COCO(ANNOTATIONS_PATH)

    ann_ids = coco.getAnnIds(imgIds=image_id, catIds=[class_id], iscrowd=None)

    image_info = coco.loadImgs(image_id)[0]

    height, width = image_info['height'], image_info['width']

    img_path = os.path.join(TRAIN_IMAGES_PATH,  image_info['file_name'])

    composite_mask  = np.zeros((height, width), dtype=np.uint8)

    if not ann_ids:
        # Reporesentamos (mascara vacía)
        if(display):
            original_image = cv2.imread(img_path)
            original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)        
            _ , axs = plt.subplots(1, 2, figsize=(10, 5))
            axs[0].imshow(original_image)
            axs[0].axis('off')
            axs[0].set_title('Imagen')
            
            axs[1].imshow(composite_mask, cmap='gray')
            axs[1].axis('off')
            axs[1].set_title('Máscara agrupando por clase')
            plt.show()        
        return composite_mask

   

    for ann in ann_ids:

        ann = coco.loadAnns(ann)[0]

        #print(ann)
           
        mask = coco.annToMask(ann)
        composite_mask = np.maximum(composite_mask, mask * 255)
    
    # Ahora representamos de forma gráfica de forma opcional

    if(display):
        original_image = cv2.imread(img_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)        
        _ , axs = plt.subplots(1, 2, figsize=(10, 5))
        axs[0].imshow(original_image)
        axs[0].axis('off')
        axs[0].set_title('Imagen')
        
        axs[1].imshow(composite_mask, cmap='gray')
        axs[1].axis('off')
        axs[1].set_title('Máscara agrupando por clase')
        plt.show()

    # La máscara la devolvemsos para guardarla como np array
    return composite_mask


In [ ]:
image_ids = set()
for cat in top_10_category_ids:
    image_ids |= set(coco.getImgIds(catIds=cat))

image_ids = list(image_ids)

In [ ]:
final_masks = []

for image_id in image_ids[:3]:

    mascaras = []

    for cat in top_10_category_ids:

        mask = mask_generator(image_id=image_id, class_id= cat, display= False )
        mascaras.append(mask)

    final_masks.append(np.stack(mascaras, axis=-1))


In [ ]:
def plot_image_with_masks(image_id, masks, categories_ids):
    # Load the image
    image_info = coco.loadImgs(image_id)[0]
    img_path = os.path.join(TRAIN_IMAGES_PATH, image_info['file_name'])

    original_image = cv2.imread(img_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # Set up the figure
    fig , axs = plt.subplots(3, 4, figsize=(15, 10))
    axs = axs.flatten()  # Flatten the array for easier indexing
    
    # Display the original image
    axs[0].imshow(original_image)
    axs[0].axis('off')
    axs[0].set_title('Image')
    
    # Plot masks
    print(f"We dhave {masks.shape[-1]}, {masks.shape}")
    for idx in range(masks.shape[-1]):
        
        # if idx >= len(axs) - 1:  # Avoid index out-of-range
        #     break

        
        mask = masks[:,:,idx]
        axs[idx + 1].imshow(mask)
        axs[idx + 1].axis('off')
        axs[idx + 1].set_title(f'{categories[categories_ids[idx]]}' if idx < len(categories) else "Unknown")
    

    
    [fig.delaxes(ax) for ax in axs.flatten() if not ax.has_data()]

    plt.tight_layout()
    plt.show()


In [ ]:
categories_ids = [cat[0] for cat in top_10_categories ]

for image, masks in zip(image_ids[:3],final_masks):



    print("Image with id", image)
    
    plot_image_with_masks(image, masks, categories_ids)
    print(" \n")

In [ ]:

image_ids = set()
for cat in top_10_category_ids:
    image_ids |= set(coco.getImgIds(catIds=cat))

image_ids = list(image_ids)


final_masks = []

for image_id in image_ids:

    # carga de la imagen
    image_info = coco.loadImgs(image_id)[0]
    img_path = os.path.join(TRAIN_IMAGES_PATH, image_info['file_name'])
    original_image = cv2.imread(img_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    mascaras = []

    # Carga y preprocesamiento de las máscaras

    for cat in top_10_category_ids:

        mask = mask_generator(image_id=image_id, class_id= cat, display= False )
        mascaras.append(mask)

    mascara_imagen = np.stack(mascaras, axis=-1)

    np_array = (original_image,mascara_imagen)
    name = str(image_id)

    save_numpy_array(np_array,name )

    # Almacenamiento de ambas
    #print(f"Imagen  {image_id} almacenada de forma exitosa")



In [ ]:
DIR_DATA_PREPROCESSED_TRAIN = os.path.join(
     os.getcwd(), "..", "data", "preprocessed_train")

for file in os.listdir(DIR_DATA_PREPROCESSED_TRAIN)[:5]:
    print(f"Fichero con nombre", file)
    ruta_imagen = os.path.join(DIR_DATA_PREPROCESSED_TRAIN, file)


    np_array = np.load(ruta_imagen)


    # Set up the figure
    fig , axs = plt.subplots(3, 4, figsize=(15, 10))
    axs = axs.flatten()  # Flatten the array for easier indexing
    
    # Display the original image
    axs[0].imshow(np_array["image"])
    axs[0].axis('off')
    axs[0].set_title('Image')
    
    # Plot masks
    masks = np_array["mask"]
    print(f"We dhave {masks.shape[-1]}, {masks.shape}")
    for idx in range(masks.shape[-1]):
        
        # if idx >= len(axs) - 1:  # Avoid index out-of-range
        #     break

        
        mask = masks[:,:,idx]
        axs[idx + 1].imshow(mask)
        axs[idx + 1].axis('off')
        axs[idx + 1].set_title(f'{categories[categories_ids[idx]]}' if idx < len(categories) else "Unknown")
    

    
    [fig.delaxes(ax) for ax in axs.flatten() if not ax.has_data()]

    plt.tight_layout()
    plt.show()


    